# Boston Housing Classification Logistic Regression

In [ ]:
from pyspark.sql.types import BooleanType
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import expr
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import DenseVector
from pyspark.mllib.evaluation import MulticlassMetrics
from helpers.helper_functions import translate_to_file_string

In [ ]:
inputFile = translate_to_file_string("../data/Boston_Housing_Data.csv")

Spark session creation 

In [ ]:
spark = (SparkSession
       .builder
       .appName("BostonHousingClass")
       .getOrCreate())

DataFrame creation using an ifered Schema 

In [ ]:
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile) \
       .withColumn("CATBOOL", expr("CAT").cast(BooleanType()))
print(df.printSchema())

In [ ]:
featureCols = df.columns.copy()
featureCols.remove("MEDV")
featureCols.remove("CAT")
featureCols.remove("CATBOOL") 
print(featureCols)

In [ ]:
assembler =  VectorAssembler(outputCol="features", inputCols=featureCols)
featureSet = assembler.transform(df)


Build Labeled Point Feature Set

In [ ]:
featureSetLP = (featureSet.select(featureSet.CAT, featureSet.features)
      .rdd
      .map(lambda row: LabeledPoint(row.CAT, DenseVector(row.features))))

Prepare training and test data.

In [ ]:
splits = featureSetLP.randomSplit([0.9, 0.1 ], 12345)
training = splits[0]
test = splits[1]

Logistic regression

Train the model 

In [ ]:
modelLRLB = LogisticRegressionWithLBFGS.train(training, numClasses=2, regParam=0, regType="l2", iterations=1000, corrections=10)

Test the model

In [ ]:
predictionAndLabels = test.map(lambda x : [float(modelLRLB.predict(x.features )), float(x.label) ])
print (predictionAndLabels.take(20))

evaluate the result

In [ ]:
metrics =  MulticlassMetrics(predictionAndLabels)
print("Test Error LRLBFG =" , (1.0 - metrics.accuracy))

In [ ]:
spark.stop()